# The Multiple_Filter object

In the [previous notebook](filter.ipynb) we looked at using the `Filter` object; now we will extend this to our `Multiple_Filter` object. This class simply holds a list of Filter objects, but makes loading these in via SVO much easier via its class methods. In addition, these filters are by default sorted from bluest to reddest based on their central wavelength, which reduces potential errors arising from manually ordering these.

We start by loading in all of the JWST filters in two different ways:
1. By looping through all band names for each of the NIRCam and MIRI instruments, making an array of `Filter` objects pulled from the SVO individually.
1. Pull all bands from each JWST imaging instrument in one go and add these together.

In [ ]:
# imports
import time
import matplotlib.pyplot as plt
from galfind import Filter, Multiple_Filter #, UVJ_Filters

In [ ]:
# Load in the filter objects

# time the load-in
start = time.time()

NIRCam_band_names = [
    "F070W", "F090W", "F115W", "F140M", "F150W", "F162M", "F164N", "F150W2",
    "F182M", "F187N", "F200W", "F210M", "F212N", "F250M", "F277W", "F300M",
    "F323N", "F322W2","F335M", "F356W", "F360M", "F405N", "F410M", "F430M",
    "F444W", "F460M", "F466N", "F470N", "F480M",
]
MIRI_band_names = [
    "F560W", "F770W", "F1000W", "F1065C", "F1140C", "F1130W", "F1280W",
    "F1500W", "F1550C", "F1800W", "F2100W", "F2300C", "F2550W",
]

JWST_filters = [Filter.from_SVO("JWST", "NIRCam", band_name) for band_name in NIRCam_band_names] + [Filter.from_SVO("JWST", "MIRI", band_name) for band_name in MIRI_band_names]
JWST_multi_filter_obj_1 = Multiple_Filter(JWST_filters)
end = time.time()
print(f"Time to load in JWST filters using method 1: {end - start}s")

In [ ]:
# load all JWST filters
start = time.time()
JWST_multi_filter_obj_2 = Multiple_Filter.from_facility("JWST")
end = time.time()
print(f"Time to load in JWST filters using method 2: {end - start}s")

# confirm that these two methods are equivalent
assert JWST_multi_filter_obj_1 == JWST_multi_filter_obj_2

Apart from (hopefully) being slightly faster, the major advantage of method 2 is that it requires no prior knowledge of which bands are included in each imaging instrument. As with the `Filter` object, the `Multiple_Filter` class also has a `plot()` method which iteratively calls the `Filter.plot()` method. Let's try using this now.

In [ ]:
# plot the filters
fig, ax = plt.subplots()
JWST_multi_filter_obj_1.plot(ax, show = True)

It is pretty obvious from the above plot that this can get busy pretty quickly. Let's try plotting just the NIRCam bands.

In [ ]:
# load and plot the NIRCam filters
NIRCam_filt = Multiple_Filter.from_instrument("NIRCam")
# plot the filters
fig, ax = plt.subplots()
NIRCam_filt.plot(ax, show = True)

#NIRCam_ACS_WFC_filters = Multiple_Filter.from_instruments(["NIRCam", "ACS_WFC"])

This is probably still a little too busy. We could try plotting just the JWST/NIRCam wide bands.

In [ ]:
# load and plot the NIRCam wideband filters
NIRCam_widebands = Multiple_Filter.from_instrument("NIRCam", keep_suffix = "W")
fig, ax = plt.subplots()
NIRCam_widebands.plot(ax, show = True)

This definitely looks neater. Let's now plot the standard 8 JWST/NIRCam filters used in many cycle 1 large programs e.g. in the [PEARLS](https://sites.google.com/view/jwstpearls) imaging dataset. To do this we could create a brand new `Multiple_Filter` object as before, but since we have already loaded in most of the `Filter` objects from SVO, we will simply use the overloaded `-` operator to remove the less sensitive F070W filter and the overloaded `+` operator to load the F410M filter. When adding or removing bands from the `Multiple_Filter` object, either a string representation or a `Filter` object can be used.

In [ ]:
f410m = Filter.from_SVO("JWST", "NIRCam", "F410M")

NIRCam_widebands_plus_f410m = NIRCam_widebands + f410m
PEARLS_filters = NIRCam_widebands_plus_f410m - "F070W"

We will now have a look at some of the other class methods which make it easier to load in the specific filters you want for your analysis. 